<a href="https://colab.research.google.com/github/ineventhorizon/AI-Course-Progress-Example-Code/blob/main/AINewsSummarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers sumy rouge-score nltk

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5077e172d73b0b3ee7128403e2e9fb10fc542a2bf0219442c39a27f83d5a29d5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=631e1d6ebc1ed830935f73bba33cd39ba30130f8d76e47d6d042c25a9e867ad0
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f2023a1e0a04b4cf1d8748e3667646b4756b0bf126ed026f0de45081e491d4ad
  Stored in directory: /roo

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

100%|██████████| 26.5M/26.5M [00:00<00:00, 61.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')  # Often used with other nltk functionalities
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from transformers import pipeline
from rouge_score import rouge_scorer

# Download required NLTK resources
nltk.download("punkt")

# User profile for personalization
user_profile = {
    "topics": ["technology", "AI"],
    "sentiment": "neutral",
    "summary_length": 5  # Number of sentences
}

# Step 1: Preprocessing the text
def preprocess_text(text):
    # Basic cleaning of text
    text = text.replace("\n", " ").strip()
    return text

# Step 2: Extractive summarization using LexRank
def extractive_summary(text, num_sentences=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join([str(sentence) for sentence in summary])

# Step 3: Abstractive summarization using transformers
def abstractive_summary(text):
    summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarization_pipeline(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]["summary_text"]

# Step 4: Personalization of the summary
def personalize_summary(summary, user_profile):
    # Filter sentences based on user preferences
    personalized_sentences = []
    for sentence in nltk.sent_tokenize(summary):
        if any(topic.lower() in sentence.lower() for topic in user_profile["topics"]):
            personalized_sentences.append(sentence)
    return " ".join(personalized_sentences[: user_profile["summary_length"]])

# Step 5: Evaluation using ROUGE
def evaluate_summary(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

def main():
    article = """
    Artificial intelligence (AI) is at the forefront of innovation, revolutionizing sectors such as healthcare, technology, and transportation. Advanced AI systems, like generative models such as GPT-4, are transforming the way industries operate. These models enable applications such as virtual assistants, content creation, and sophisticated data analysis, providing unprecedented efficiency and scalability.

In healthcare, AI is improving patient outcomes by powering diagnostics, predictive analytics, and personalized treatment plans. Similarly, in technology, AI is accelerating software development and enabling intelligent automation. The transportation sector is witnessing the emergence of self-driving cars and optimized logistics systems driven by AI.

However, alongside these advancements, ethical concerns loom large. Issues such as algorithmic bias, data privacy, and misuse of AI technologies present significant challenges. For instance, biased algorithms can reinforce existing societal inequities, while the misuse of generative models raises concerns about misinformation and security risks. Addressing these challenges requires robust frameworks for transparency, fairness, and accountability in AI development and deployment.

As AI continues to evolve, it remains a double-edged sword: a tool for transformative progress and a source of profound ethical dilemmas. Striking the right balance will be key to ensuring that AI serves as a force for good in society.
    """
    # Reference summary for evaluation
    reference_summary = """
    AI is revolutionizing industries such as healthcare, technology, and transportation with advancements like GPT-4, which enable applications such as diagnostics, automation, and virtual assistants. While these developments promise significant benefits, they are accompanied by ethical challenges, including algorithmic bias and misuse concerns. Balancing innovation with accountability is essential for the responsible evolution of AI.
    """

    # Process the article
    clean_article = preprocess_text(article)

    # Generate summaries
    extractive = extractive_summary(clean_article, num_sentences=user_profile["summary_length"])
    abstractive = abstractive_summary(clean_article)

    # Combine both summaries for a hybrid approach
    hybrid_summary = f"Extractive Summary:\n{extractive}\n\nAbstractive Summary:\n{abstractive}"

    # Personalize the summary
    personalized_summary = personalize_summary(abstractive, user_profile)

    # Evaluate the summary
    scores = evaluate_summary(reference_summary, personalized_summary)

    # Output results
    print("Hybrid Summary:\n", hybrid_summary)
    print("\nPersonalized Summary:\n", personalized_summary)
    print("\nEvaluation Scores (ROUGE):\n", scores)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hybrid Summary:
 Extractive Summary:
Artificial intelligence (AI) is at the forefront of innovation, revolutionizing sectors such as healthcare, technology, and transportation. Advanced AI systems, like generative models such as GPT-4, are transforming the way industries operate. These models enable applications such as virtual assistants, content creation, and sophisticated data analysis, providing unprecedented efficiency and scalability. In healthcare, AI is improving patient outcomes by powering diagnostics, predictive analytics, and personalized treatment plans. Similarly, in technology, AI is accelerating software development and enabling intelligent automation.

Abstractive Summary:
Artificial intelligence (AI) is at the forefront of innovation, revolutionizing sectors such as healthcare, technology, and transportation. However, alongside these advancements, ethical concerns loom large. Issues such as algorithmic bias, data privacy, and misuse of AI technologies present signific